In [2]:
%pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from torchvision import transforms
from PIL import Image
import pandas as pd
import os

In [3]:
clean_data = pd.read_csv("dataset/dataset_tokenized.csv")
clean_data.head()

,tweet_text,image_path,labels,input_ids,attention_mask
0,rt xxsugvngxx i ran into this holy nigga today,dataset/images/0.jpg,"[1, 0, 0]","tensor([ 101, 19387, 22038, 6342, 2290, 160...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1,everybody calling you nigger now,dataset/images/1.jpg,"[1, 0, 1]","tensor([ 101, 7955, 4214, 2017, 9152, 133...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,..."
2,fuck ice white supremacist trash all of you ar...,dataset/images/2.jpg,"[5, 1, 1]","tensor([ 101, 6616, 3256, 2317, 10514, 281...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
3,sissy faggot,dataset/images/3.jpg,"[3, 3, 0]","tensor([ 101, 24761, 6508, 6904, 13871, 41...","tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,..."
4,nigga what,dataset/images/4.jpg,"[4, 0, 3]","tensor([ 101, 9152, 23033, 2054, 102, ...","tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [4]:
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import torch

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for ResNet-50
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(           # Normalize using ImageNet stats
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Function to load and transform images with validation
def load_and_transform_image(image_path):
    try:
        # Ensure the image path is absolute
        if not os.path.isabs(image_path):
            # Construct the absolute path
            base_dir = ""
            image_path = os.path.join(base_dir, image_path)
        
        # Open and convert the image to RGB
        image = Image.open(image_path).convert("RGB")
        
        # Apply transformations
        transformed_image = transform(image)
        
        # Validate the transformed image
        if not isinstance(transformed_image, torch.Tensor):
            raise ValueError("Transformed image is not a tensor.")
        if transformed_image.shape != (3, 224, 224):
            raise ValueError(f"Unexpected image shape: {transformed_image.shape}")
        
        # Check if pixel values are within a reasonable range after normalization
        if transformed_image.min() < -3 or transformed_image.max() > 3:
            raise ValueError("Image pixel values are outside the expected range after normalization.")
        
        print(f"Successfully transformed image: {image_path}")
        return transformed_image
    
    except Exception as e:
        print(f"Error loading or transforming image {image_path}: {e}")
        return transform(Image.new("RGB", (224, 224), color="black"))  # Return a black image if loading fails

# Apply image transformations to the dataset
clean_data["image"] = clean_data["image_path"].apply(load_and_transform_image)

# Save transformed images and update the dataset
transformed_images_dir = "dataset/transformed_images"
os.makedirs(transformed_images_dir, exist_ok=True)

# Save transformed images and store their paths
image_paths = []
for idx, row in clean_data.iterrows():
    image_tensor = row["image"]
    image_name = f"{idx}.pt"  # Save as .pt file
    image_path = os.path.join(transformed_images_dir, image_name)
    torch.save(image_tensor, image_path)  # Save the tensor
    image_paths.append(image_path)

# Add the transformed image paths to the DataFrame
clean_data["transformed_image_path"] = image_paths

# Drop the "image" column (tensors cannot be saved to CSV)
clean_data = clean_data.drop(columns=["image"])

# Save the updated dataset to a CSV file
clean_data.to_csv("dataset/dataset_transformed.csv", index=False)
print("Dataset saved to dataset/dataset_transformed.csv")

Successfully transformed image: dataset/images/0.jpg
Successfully transformed image: dataset/images/1.jpg
Successfully transformed image: dataset/images/2.jpg
Successfully transformed image: dataset/images/3.jpg
Successfully transformed image: dataset/images/4.jpg
Successfully transformed image: dataset/images/5.jpg
Successfully transformed image: dataset/images/6.jpg
Successfully transformed image: dataset/images/7.jpg
Successfully transformed image: dataset/images/8.jpg
Successfully transformed image: dataset/images/9.jpg
Successfully transformed image: dataset/images/10.jpg
Successfully transformed image: dataset/images/11.jpg
Successfully transformed image: dataset/images/12.jpg
Successfully transformed image: dataset/images/13.jpg
Successfully transformed image: dataset/images/14.jpg
Successfully transformed image: dataset/images/15.jpg
Successfully transformed image: dataset/images/16.jpg
Successfully transformed image: dataset/images/17.jpg
Successfully transformed image: datase